# 01 - Collecte de Données : Ligue 1

## Phase 0 — Validation du Modèle

**Objectif :** Récupérer les résultats de matchs terminés pour la Ligue 1 via l'API Football-Data.org.

Nous collectons deux saisons :
- **2023-24** → Données d'entraînement (training set)
- **2024-25** → Données de test walk-forward (out-of-sample)

Cette séparation temporelle est cruciale : on entraîne sur le passé et on valide sur le futur,
exactement comme on ferait en production.

### Sources de données
| Source | Usage | Quota |
|--------|-------|-------|
| Football-Data.org | Scores, résultats, journées | 100 req/jour (gratuit) |
| The Odds API | Cotes de bookmakers (optionnel) | 500 req/mois (gratuit) |

---
## 1. Configuration et imports

In [1]:
import os
import sys
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

# --- Chemins du projet ---
# On remonte d'un niveau depuis notebooks/ pour atteindre la racine du projet
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"

# Créer le dossier raw/ s'il n'existe pas
RAW_DIR.mkdir(parents=True, exist_ok=True)

# --- Ajouter src/ au path pour importer nos modules ---
SRC_DIR = str(PROJECT_ROOT / "src")
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

# --- Charger les variables d'environnement ---
load_dotenv(PROJECT_ROOT / ".env")

print(f"Racine du projet : {PROJECT_ROOT}")
print(f"Dossier données brutes : {RAW_DIR}")
print(f"Clé Football-Data.org : {'✓ configurée' if os.getenv('FOOTBALL_DATA_ORG_KEY') else '✗ manquante'}")
print(f"Clé Odds API : {'✓ configurée' if os.getenv('ODDS_API_KEY') else '✗ manquante (optionnel)'}")

Racine du projet : /Users/hatemahmed/football-predictions
Dossier données brutes : /Users/hatemahmed/football-predictions/data/raw
Clé Football-Data.org : ✓ configurée
Clé Odds API : ✓ configurée


In [2]:
# Import du client Football-Data.org depuis src/data/
from data.football_data_org import FootballDataClient

# Initialisation du client
api_key = os.getenv("FOOTBALL_DATA_ORG_KEY")
if not api_key:
    raise ValueError(
        "La clé FOOTBALL_DATA_ORG_KEY est manquante.\n"
        "Créez un fichier .env à la racine du projet avec :\n"
        "FOOTBALL_DATA_ORG_KEY=votre_clé\n"
        "Inscription gratuite : https://www.football-data.org/"
    )

client = FootballDataClient(api_key=api_key)
print("Client Football-Data.org initialisé.")

Client Football-Data.org initialisé.


---
## 2. Collecte des matchs Ligue 1 2023-24 (entraînement)

On récupère tous les matchs terminés (status=FINISHED) de la saison 2023-24.
Le paramètre `season=2023` correspond à la saison qui commence en 2023 (i.e. 2023-24).

In [3]:
# Récupération des matchs terminés 2023-24
matches_2023 = client.get_finished_matches_with_scores(
    league="ligue_1",
    season=2023,
)

df_2023 = pd.DataFrame(matches_2023)
print(f"Matchs récupérés pour 2023-24 : {len(df_2023)}")
df_2023.head()

2026-02-07 12:32:12.575 | INFO     | data.football_data_org:get_matches:72 - Fetched 306 matches for ligue_1 2023


2026-02-07 12:32:12.576 | INFO     | data.football_data_org:get_finished_matches_with_scores:102 - Got 306 finished matches with scores


Matchs récupérés pour 2023-24 : 306


,home_team,away_team,home_score,away_score,kickoff,matchday
0,OGC Nice,Lille OSC,1,1,2023-08-11T19:00:00Z,1
1,Olympique de Marseille,Stade de Reims,2,1,2023-08-12T15:00:00Z,1
2,Paris Saint-Germain FC,FC Lorient,0,0,2023-08-12T19:00:00Z,1
3,Stade Brestois 29,Racing Club de Lens,3,2,2023-08-13T11:00:00Z,1
4,Clermont Foot 63,AS Monaco FC,2,4,2023-08-13T13:00:00Z,1


In [4]:
# Nettoyage et typage
df_2023["kickoff"] = pd.to_datetime(df_2023["kickoff"])
df_2023["home_score"] = df_2023["home_score"].astype(int)
df_2023["away_score"] = df_2023["away_score"].astype(int)
df_2023["matchday"] = df_2023["matchday"].astype(int)

# Tri chronologique
df_2023 = df_2023.sort_values("kickoff").reset_index(drop=True)

# Colonnes dérivées utiles
df_2023["total_goals"] = df_2023["home_score"] + df_2023["away_score"]
df_2023["result"] = df_2023.apply(
    lambda r: "H" if r["home_score"] > r["away_score"]
    else ("A" if r["home_score"] < r["away_score"] else "D"),
    axis=1,
)

print(f"Période : {df_2023['kickoff'].min().date()} → {df_2023['kickoff'].max().date()}")
print(f"Journées : {df_2023['matchday'].min()} à {df_2023['matchday'].max()}")
df_2023.head(10)

Période : 2023-08-11 → 2024-05-19
Journées : 1 à 34


,home_team,away_team,home_score,away_score,kickoff,matchday,total_goals,result
0,OGC Nice,Lille OSC,1,1,2023-08-11 19:00:00+00:00,1,2,D
1,Olympique de Marseille,Stade de Reims,2,1,2023-08-12 15:00:00+00:00,1,3,H
2,Paris Saint-Germain FC,FC Lorient,0,0,2023-08-12 19:00:00+00:00,1,0,D
3,Stade Brestois 29,Racing Club de Lens,3,2,2023-08-13 11:00:00+00:00,1,5,H
4,Clermont Foot 63,AS Monaco FC,2,4,2023-08-13 13:00:00+00:00,1,6,A
5,FC Nantes,Toulouse FC,1,2,2023-08-13 13:00:00+00:00,1,3,A
6,Montpellier HSC,Le Havre AC,2,2,2023-08-13 13:00:00+00:00,1,4,D
7,Stade Rennais FC 1901,FC Metz,5,1,2023-08-13 15:05:00+00:00,1,6,H
8,RC Strasbourg Alsace,Olympique Lyonnais,2,1,2023-08-13 18:45:00+00:00,1,3,H
9,FC Metz,Olympique de Marseille,2,2,2023-08-18 19:00:00+00:00,2,4,D


---
## 3. Collecte des matchs Ligue 1 2024-25 (test walk-forward)

Même chose pour la saison en cours. On ne garde que les matchs déjà joués.

In [5]:
# Récupération des matchs terminés 2024-25
matches_2024 = client.get_finished_matches_with_scores(
    league="ligue_1",
    season=2024,
)

df_2024 = pd.DataFrame(matches_2024)
print(f"Matchs récupérés pour 2024-25 : {len(df_2024)}")
df_2024.head()

2026-02-07 12:32:19.637 | INFO     | data.football_data_org:get_matches:72 - Fetched 305 matches for ligue_1 2024


2026-02-07 12:32:19.638 | INFO     | data.football_data_org:get_finished_matches_with_scores:102 - Got 305 finished matches with scores


Matchs récupérés pour 2024-25 : 305


,home_team,away_team,home_score,away_score,kickoff,matchday
0,Le Havre AC,Paris Saint-Germain FC,1,4,2024-08-16T18:45:00Z,1
1,Stade Brestois 29,Olympique de Marseille,1,5,2024-08-17T15:00:00Z,1
2,Stade de Reims,Lille OSC,0,2,2024-08-17T17:00:00Z,1
3,AS Monaco FC,AS Saint-Étienne,1,0,2024-08-17T19:00:00Z,1
4,AJ Auxerre,OGC Nice,2,1,2024-08-18T13:00:00Z,1


In [6]:
# Nettoyage et typage (même traitement que 2023-24)
df_2024["kickoff"] = pd.to_datetime(df_2024["kickoff"])
df_2024["home_score"] = df_2024["home_score"].astype(int)
df_2024["away_score"] = df_2024["away_score"].astype(int)
df_2024["matchday"] = df_2024["matchday"].astype(int)

df_2024 = df_2024.sort_values("kickoff").reset_index(drop=True)

df_2024["total_goals"] = df_2024["home_score"] + df_2024["away_score"]
df_2024["result"] = df_2024.apply(
    lambda r: "H" if r["home_score"] > r["away_score"]
    else ("A" if r["home_score"] < r["away_score"] else "D"),
    axis=1,
)

print(f"Période : {df_2024['kickoff'].min().date()} → {df_2024['kickoff'].max().date()}")
print(f"Journées : {df_2024['matchday'].min()} à {df_2024['matchday'].max()}")
df_2024.head(10)

Période : 2024-08-16 → 2025-05-17
Journées : 1 à 34


,home_team,away_team,home_score,away_score,kickoff,matchday,total_goals,result
0,Le Havre AC,Paris Saint-Germain FC,1,4,2024-08-16 18:45:00+00:00,1,5,A
1,Stade Brestois 29,Olympique de Marseille,1,5,2024-08-17 15:00:00+00:00,1,6,A
2,Stade de Reims,Lille OSC,0,2,2024-08-17 17:00:00+00:00,1,2,A
3,AS Monaco FC,AS Saint-Étienne,1,0,2024-08-17 19:00:00+00:00,1,1,H
4,AJ Auxerre,OGC Nice,2,1,2024-08-18 13:00:00+00:00,1,3,H
5,Montpellier HSC,RC Strasbourg Alsace,1,1,2024-08-18 15:00:00+00:00,1,2,D
6,Toulouse FC,FC Nantes,0,0,2024-08-18 15:00:00+00:00,1,0,D
7,Angers SCO,Racing Club de Lens,0,1,2024-08-18 15:00:00+00:00,1,1,A
8,Stade Rennais FC 1901,Olympique Lyonnais,3,0,2024-08-18 18:45:00+00:00,1,3,H
9,Paris Saint-Germain FC,Montpellier HSC,6,0,2024-08-23 18:45:00+00:00,2,6,H


In [7]:
# Nombre de requêtes API utilisées jusqu'ici
print(f"Requêtes API utilisées : {client.requests_used} / 100 (quota journalier)")

Requêtes API utilisées : 2 / 100 (quota journalier)


---
## 4. Sauvegarde en CSV

On sauvegarde les données brutes dans `data/raw/` pour ne pas
avoir à refaire les appels API à chaque exécution.

In [8]:
# Sauvegarde des CSV
path_2023 = RAW_DIR / "ligue1_2023.csv"
path_2024 = RAW_DIR / "ligue1_2024.csv"

df_2023.to_csv(path_2023, index=False)
df_2024.to_csv(path_2024, index=False)

print(f"Sauvegardé : {path_2023} ({len(df_2023)} lignes, {path_2023.stat().st_size / 1024:.1f} Ko)")
print(f"Sauvegardé : {path_2024} ({len(df_2024)} lignes, {path_2024.stat().st_size / 1024:.1f} Ko)")

Sauvegardé : /Users/hatemahmed/football-predictions/data/raw/ligue1_2023.csv (306 lignes, 20.3 Ko)
Sauvegardé : /Users/hatemahmed/football-predictions/data/raw/ligue1_2024.csv (305 lignes, 20.4 Ko)


---
## 5. Statistiques descriptives

Vérifions que les données sont cohérentes avec ce qu'on attend de la Ligue 1.

In [9]:
def afficher_stats(df: pd.DataFrame, saison: str) -> None:
    """Affiche les statistiques clés d'une saison."""
    n = len(df)
    print(f"{'=' * 50}")
    print(f"  LIGUE 1 {saison}")
    print(f"{'=' * 50}")
    print(f"  Nombre de matchs       : {n}")
    print(f"  Équipes distinctes     : {df['home_team'].nunique()}")
    print()

    # Répartition des résultats
    counts = df["result"].value_counts()
    print("  Répartition des résultats :")
    for label, full_name in [("H", "Victoire domicile"), ("D", "Match nul"), ("A", "Victoire extérieur")]:
        c = counts.get(label, 0)
        pct = c / n * 100
        print(f"    {full_name:25s} : {c:3d}  ({pct:5.1f}%)")
    print()

    # Buts
    print("  Statistiques de buts :")
    print(f"    Moyenne buts/match    : {df['total_goals'].mean():.2f}")
    print(f"    Moyenne buts domicile : {df['home_score'].mean():.2f}")
    print(f"    Moyenne buts extérieur: {df['away_score'].mean():.2f}")
    print(f"    Matchs > 2.5 buts    : {(df['total_goals'] > 2.5).sum():3d}  ({(df['total_goals'] > 2.5).mean() * 100:.1f}%)")
    print(f"    Matchs 0-0           : {((df['home_score'] == 0) & (df['away_score'] == 0)).sum()}")
    print()

In [10]:
afficher_stats(df_2023, "2023-24")
afficher_stats(df_2024, "2024-25")

  LIGUE 1 2023-24
  Nombre de matchs       : 306
  Équipes distinctes     : 18

  Répartition des résultats :
    Victoire domicile         : 120  ( 39.2%)
    Match nul                 :  81  ( 26.5%)
    Victoire extérieur        : 105  ( 34.3%)

  Statistiques de buts :
    Moyenne buts/match    : 2.70
    Moyenne buts domicile : 1.45
    Moyenne buts extérieur: 1.25
    Matchs > 2.5 buts    : 162  (52.9%)
    Matchs 0-0           : 27

  LIGUE 1 2024-25
  Nombre de matchs       : 305
  Équipes distinctes     : 18

  Répartition des résultats :
    Victoire domicile         : 143  ( 46.9%)
    Match nul                 :  62  ( 20.3%)
    Victoire extérieur        : 100  ( 32.8%)

  Statistiques de buts :
    Moyenne buts/match    : 2.98
    Moyenne buts domicile : 1.61
    Moyenne buts extérieur: 1.37
    Matchs > 2.5 buts    : 170  (55.7%)
    Matchs 0-0           : 14



In [11]:
# Vérification rapide : une saison Ligue 1 complète = 306 matchs (18 x 17)
# Depuis 2023-24 la Ligue 1 a 18 équipes au lieu de 20
EXPECTED_FULL_SEASON = 306

if len(df_2023) == EXPECTED_FULL_SEASON:
    print(f"2023-24 : saison complète ({EXPECTED_FULL_SEASON} matchs) ✓")
else:
    print(f"2023-24 : {len(df_2023)} / {EXPECTED_FULL_SEASON} matchs")

if len(df_2024) == EXPECTED_FULL_SEASON:
    print(f"2024-25 : saison complète ({EXPECTED_FULL_SEASON} matchs) ✓")
else:
    print(f"2024-25 : {len(df_2024)} / {EXPECTED_FULL_SEASON} matchs (saison en cours)")

2023-24 : saison complète (306 matchs) ✓
2024-25 : 305 / 306 matchs (saison en cours)


---
## 6. (Optionnel) Récupération des cotes — The Odds API

Si la clé `ODDS_API_KEY` est configurée, on récupère les cotes actuelles
pour les matchs à venir. Ces cotes serviront de benchmark : notre modèle
doit battre les cotes implicites des bookmakers pour avoir de la valeur.

**Note :** The Odds API ne fournit que les cotes des matchs à venir,
pas les cotes historiques. Pour les cotes historiques (nécessaires en
backtesting), il faudra utiliser une autre source ou les collecter au fil du temps.

In [12]:
odds_api_key = os.getenv("ODDS_API_KEY")

if not odds_api_key:
    print("Clé ODDS_API_KEY non configurée — cette section est ignorée.")
    print("Pour récupérer les cotes, ajoutez ODDS_API_KEY dans votre .env")
    print("Inscription : https://the-odds-api.com/")
else:
    from data.odds_api import OddsAPIClient, extract_best_odds, remove_margin

    odds_client = OddsAPIClient(api_key=odds_api_key)

    # Récupérer les cotes pour les matchs à venir de Ligue 1
    try:
        odds_data = odds_client.get_odds(
            league="ligue_1",
            markets="h2h,totals",
            regions="eu",
        )

        if odds_data:
            print(f"Cotes récupérées pour {len(odds_data)} matchs à venir.")
            print(f"Requêtes restantes : {odds_client.remaining_requests}")
            print()

            # Afficher les meilleures cotes pour chaque match
            rows = []
            for match in odds_data:
                best = extract_best_odds(match)
                fair = remove_margin(
                    best["home"]["odds"],
                    best["draw"]["odds"],
                    best["away"]["odds"],
                )
                rows.append({
                    "home_team": match.get("home_team", ""),
                    "away_team": match.get("away_team", ""),
                    "home_odds": best["home"]["odds"],
                    "draw_odds": best["draw"]["odds"],
                    "away_odds": best["away"]["odds"],
                    "over25_odds": best["over25"]["odds"],
                    "under25_odds": best["under25"]["odds"],
                    "fair_home": round(fair["home"], 3),
                    "fair_draw": round(fair["draw"], 3),
                    "fair_away": round(fair["away"], 3),
                    "overround": round(fair["overround"], 3),
                })

            df_odds = pd.DataFrame(rows)

            # Sauvegarder les cotes
            odds_path = RAW_DIR / "ligue1_odds_current.csv"
            df_odds.to_csv(odds_path, index=False)
            print(f"Sauvegardé : {odds_path}")
            print()
            display(df_odds)
        else:
            print("Aucun match à venir trouvé (hors saison ou trêve).")

        odds_client.close()

    except Exception as e:
        print(f"Erreur lors de la récupération des cotes : {e}")
        odds_client.close()

2026-02-07 12:32:20.053 | INFO     | data.odds_api:get_odds:63 - Fetched odds for 22 matches (ligue_1). Remaining: 487


Cotes récupérées pour 22 matchs à venir.
Requêtes restantes : 487

Sauvegardé : /Users/hatemahmed/football-predictions/data/raw/ligue1_odds_current.csv



,home_team,away_team,home_odds,draw_odds,away_odds,over25_odds,under25_odds,fair_home,fair_draw,fair_away,overround
0,RC Lens,Rennes,1.72,4.50,5.70,1.62,2.45,0.594,0.227,0.179,0.979
1,Brest,Lorient,2.46,3.30,3.45,2.26,1.78,0.407,0.303,0.290,0.999
2,Nantes,Lyon,5.20,3.95,1.83,2.10,1.95,0.194,0.255,0.551,0.992
3,Nice,AS Monaco,3.38,3.95,2.20,1.53,2.40,0.295,0.252,0.453,1.004
4,Angers,Toulouse,3.90,3.27,2.26,2.52,1.64,0.255,0.304,0.440,1.005
5,Auxerre,Paris FC,2.60,3.40,3.10,2.24,1.76,0.384,0.294,0.322,1.001
6,Le Havre,Strasbourg,4.50,3.80,1.94,2.02,1.93,0.222,0.263,0.515,1.001
7,Paris Saint Germain,Marseille,1.48,5.50,7.00,1.40,2.75,0.675,0.182,0.143,1.000
8,Rennes,Paris Saint Germain,6.04,4.93,1.54,0.00,0.00,0.163,0.199,0.638,1.018
9,AS Monaco,Nantes,1.50,5.03,7.01,1.44,2.50,0.661,0.197,0.142,1.008


---
## 7. Fermeture et résumé

On ferme proprement le client HTTP et on affiche un récapitulatif.

In [13]:
# Fermer le client HTTP
client.close()

print("Résumé de la collecte")
print("=" * 40)
print(f"Fichiers générés :")
for f in sorted(RAW_DIR.glob("ligue1_*.csv")):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:30s} ({size_kb:.1f} Ko)")
print()
print(f"Requêtes API consommées : {client.requests_used}")
print()
print("Prochaine étape : 02_feature_engineering.ipynb")

Résumé de la collecte
Fichiers générés :
  ligue1_2023.csv                (20.3 Ko)
  ligue1_2024.csv                (20.4 Ko)
  ligue1_odds_current.csv        (1.5 Ko)

Requêtes API consommées : 2

Prochaine étape : 02_feature_engineering.ipynb
